In [ ]:
!pip install nats_bench
!pip install xautodl

In [ ]:
import tarfile
#from xautodl.datasets.DownsampledImageNet import ImageNet16
from nats_bench import create
!wget 'https://www.dropbox.com/s/pasubh1oghex3g9/?dl=1' -O 'NATS-tss-v1_0-3ffb9-simple.tar'
#!wget 'https://www.dropbox.com/s/o2fg17ipz57nru1/?dl=1' -O ImageNet16.tar.gz
!tar xvf "NATS-tss-v1_0-3ffb9-simple.tar"

In [ ]:
!wget 'https://www.dropbox.com/s/o2fg17ipz57nru1/?dl=1' -O ImageNet16.tar.gz
file = tarfile.open('ImageNet16.tar.gz')
file.extractall('.')
file.close()

# IMPORTS and CREATION of the API instance for the topology search space (TSS) in NATS-Bench

In [20]:

import numpy as np, collections
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from xautodl.models import get_cell_based_tiny_net, get_search_spaces, CellStructure, get_search_spaces
from xautodl.utils import get_model_infos, obtain_accuracy
from xautodl.datasets.DownsampledImageNet import ImageNet16
import random
import os
import copy
import math
from scipy import stats
import time
import collections
import os, sys, time, glob, random, argparse
from copy import deepcopy
from collections import defaultdict

from nats_bench import create, genotype_utils
api = create("/content/NATS-tss-v1_0-3ffb9-simple", 'tss', fast_mode=True, verbose=False)
assert torch.cuda.is_available(), "CUDA is not available."
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
batch_size = 128


# Settings for trainloader and data argumentation 

In [ ]:
## https://github.com/D-X-Y/AutoDL-Projects/blob/f46486e21b71ae6459a700be720d7648b5429569/xautodl/datasets/get_dataset_with_transform.py#L99
def get_datasets(name):
    if name == "cifar10":
        mean = [x / 255 for x in [125.3, 123.0, 113.9]]
        std = [x / 255 for x in [63.0, 62.1, 66.7]]
    elif name == "cifar100":
        mean = [x / 255 for x in [129.3, 124.1, 112.4]]
        std = [x / 255 for x in [68.2, 65.4, 70.4]]
    elif name.startswith("ImageNet16"):
        mean = [x / 255 for x in [122.68, 116.66, 104.01]]
        std = [x / 255 for x in [63.22, 61.26, 65.09]]
    else:
        raise TypeError("Unknown dataset : {:}".format(name))

    # Data Argumentation
    if name == "cifar10" or name == "cifar100":
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean, std),
        ])
    elif name.startswith("ImageNet16"):
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean, std),
        ])
    
    if name == "cifar10": 
      trainset = dset.CIFAR10("/content/Cifar10", train=True, transform = transform, download=True)
    elif name == "cifar100": 
      trainset = dset.CIFAR100("/content/Cifar100", train=True ,transform = transform, download=True)
    elif name.startswith("ImageNet16"): 
      trainset = ImageNet16("ImageNet16", train=True, transform = transform)
    else:
      raise TypeError("Unknown dataset : {:}".format(name))
  
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,shuffle=True, num_workers=0, pin_memory = True)
    return trainloader
datasets = ["cifar10", "cifar100", "ImageNet16-120"]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
result = pd.DataFrame(columns=['Dataset', 'Network_index', 'Metric', 'Accuracy', 'Time'])


# SYNFLOW

In [ ]:
rm -rf /content/zerocostnas

In [6]:
!git clone https://github.com/SamsungLabs/zero-cost-nas.git
import os
os.rename("zero-cost-nas", "zerocostnas")
from zerocostnas.foresight.models import nasbench2, nasbench2_ops
from zerocostnas.foresight.pruners import p_utils
from zerocostnas.foresight.pruners.measures import *
from zerocostnas.foresight.pruners import predictive
###SYNFLOW from github
#rename folder zero-cost-nas
import torch.nn as nn
import torch.nn.functional as F

def synflow(arch_str, dataset, train_loader, device, dataload='random', dataload_info=1):
  start = time.time()
  def get_nclasses(dataset):
    return 100 if dataset == 'cifar100' else 10 if dataset == 'cifar10' else 120
  # We use the NAS-Bench-201 function because NATS-Bench is an extension of NAS-Bench-201
  # To use Zero-Cost-Proxies functions as synflow, we need the entire model of the network;
  # that's why we use the class model of NAS-Bench-201
  net = nasbench2.get_model_from_arch_str(arch_str.tostr(), get_nclasses(dataset))
  measures = predictive.find_measures(net, train_loader, (dataload, dataload_info, get_nclasses(dataset)), device, measure_names=['synflow'])
  # We decide to not use only synflow as a result, but normalizing it thanks to the natural logarithm of
  # (1 + (measures['synflow'])), base e
  return math.log1p(measures['synflow']), time.time() - start 


Cloning into 'zero-cost-nas'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 54 (delta 11), reused 45 (delta 10), pack-reused 0
Unpacking objects: 100% (54/54), done.


In [ ]:
net = nasbench2.get_model_from_arch_str('|nor_conv_3x3~0|+|nor_conv_3x3~0|nor_conv_3x3~1|+|skip_connect~0|nor_conv_1x1~1|nor_conv_3x3~2|', 10)
print(net)

# REA with NASWOT without modifications

In [ ]:
def counting_forward_hook(module, inp, out):
  if isinstance(inp, tuple):
      inp = inp[0]
  inp = inp.view(inp.size(0), -1)
  x = (inp > 0).float()
  # K matrix is not calcuated with Hamming Distance 
  K = x @ x.t()
  K2 = (1.-x) @ (1.-x.t())
  global Ktemp
  Ktemp = Ktemp + K.cpu().numpy() + K2.cpu().numpy()

def init(m):
    if isinstance(m, (torch.nn.Conv2d, torch.nn.Linear)):
        torch.nn.init.xavier_normal_(m.weight)

# this is the logarithm of the determinant of K 
def hooklogdet(Ktemp, labels=None):
  s, ld = np.linalg.slogdet(Ktemp)
  return ld
  
def kernel(arch, dataset, batch_size, api, train_loader):
    start = time.time()
    index = api.query_index_by_arch(arch.tostr())
    config = api.get_net_config(index, dataset)  
    
    network = get_cell_based_tiny_net(config)
    del config
    network.apply(init)

    global Ktemp
    Ktemp = np.zeros((batch_size,batch_size))
    for name, module in network.named_modules():
      if 'ReLU' in str(type(module)):
        module.register_forward_hook(counting_forward_hook)
    
    network = network.to(device)
    data_iterator = iter(train_loader)
    x, target = next(data_iterator)
    x, target = x.to(device), target.to(device)
    network(x)
    logdet= hooklogdet(Ktemp, target)
    #print("logdet: ", logdet)
    return logdet, time.time() - start

def random_topology_func(op_names, max_nodes=4):
  # Return a random architecture
  def random_architecture():
      genotypes = []
      for i in range(1, max_nodes):
        xlist = []
        for j in range(i):
          node_str = "{:}<-{:}".format(i, j)
          op_name = random.choice(op_names)
          xlist.append((op_name, j))
        genotypes.append(tuple(xlist))
      
      return CellStructure(genotypes)
      
  return random_architecture

def mutate_topology_func(op_names):
    """Computes the architecture for a child of the given parent architecture.
    The parent architecture is cloned and mutated to produce the child architecture. 
    The child architecture is mutated by randomly switch one operation to another.
    """
    def mutate_architecture(parent_arch):
        child_arch = deepcopy(parent_arch)
        node_id = random.randint(0, len(child_arch.nodes) - 1)
        node_info = list(child_arch.nodes[node_id])
        snode_id = random.randint(0, len(node_info) - 1)
        xop = random.choice(op_names)
        while xop == node_info[snode_id][0]:
            xop = random.choice(op_names)
        node_info[snode_id] = (xop, node_info[snode_id][1])
        child_arch.nodes[node_id] = tuple(node_info)
        return child_arch

    return mutate_architecture

model = { "arch": None, "synflow": None}
child = { "arch": None, "synflow": None}

def regularized_evolution_kernel(
    cycles,
    population_size,
    sample_size,
    random_arch,
    mutate_arch,
    api,
    dataset,
    train_loader
):
    """Algorithm for regularized evolution (i.e. aging evolution).
    Follows "Algorithm 1" in Real et al. "Regularized Evolution for Image
    Classifier Architecture Search".
    Args:
      cycles: the number of cycles the algorithm should run for.
      population_size: the number of individuals to keep in the population.
      sample_size: the number of individuals that should participate in each tournament.
      time_budget: the upper bound of searching cost
    Returns:
      history: a list of `Model` instances, representing all the models computed
          during the evolution experiment.
    """
    population = []
    history, total_time_cost = (
        [],
        [],
    )  # Not used by the algorithm, only used to report results.
    current_best_index = []
    # Initialize the population with random models.
    while len(population) < population_size:
        model["arch"] = random_arch()
        model["metric"], total_cost = kernel( 
            model["arch"], dataset, batch_size, api, train_loader #if use_proxy else api.full_train_epochs
        )
        # Append the info
        population.append((model["metric"], model["arch"]))
        history.append((model["metric"], model["arch"]))
        #print(history)
        # print(max(history, key=lambda x: x[0]))
        total_time_cost.append(total_cost)
        current_best_index.append(
            api.query_index_by_arch(max(history, key=lambda x: x[0])[1])
        )
    
    # Carry out evolution in cycles. Each cycle produces a model and removes another.
    while len(history) < cycles:
        # Sample randomly chosen models from the current population.
        start_time, sample = time.time(), []
        while len(sample) < sample_size:
            # Inefficient, but written this way for clarity. In the case of neural
            # nets, the efficiency of this line is irrelevant because training neural
            # nets is the rate-determining step.
            candidate = random.choice(list(population))
            sample.append(candidate)
        # The parent is the best model in the sample.
   
        parent = max(sample, key=lambda i: i[0])
  
        # Create the child model and store it.
        child["arch"] = mutate_arch(parent[1])

        child["metric"], total_cost = kernel( 
            child["arch"], dataset, batch_size, api, train_loader #if use_proxy else api.full_train_epochs
        )
        #print("c:\n", child['arch'], child['metric'])
        # Append the info
        population[:-1] = population[1:] ##popleft
        population.append((child["metric"], child["arch"]))
        #print('pop after:\n', population)
        history.append((child["metric"], child["arch"]))
        current_best_index.append(
            api.query_index_by_arch(max(history, key=lambda x: x[0])[1])
        )
        total_time_cost.append(total_cost)
        del sample, candidate, parent

    return history, current_best_index, total_time_cost

search_space = get_search_spaces("tss", "nats-bench") 
random_arch = random_topology_func(op_names = search_space) 
mutate_arch = mutate_topology_func(search_space)

## To obtain RESULTS

In [ ]:
Ktemp = []
for dataset in datasets:
  for run in range(15):
    train_loader = get_datasets(dataset)
    ## REA che ritorna  history e current_best_index
    history, current_best_index, total_times = regularized_evolution_kernel(cycles = 200, ## in history = ((acc, arch), (acc2, arch2)....)
                                                                    population_size = 50, 
                                                                    sample_size = 10, 
                                                                    random_arch= random_arch, 
                                                                    mutate_arch=mutate_arch,  
                                                                    api=api, 
                                                                    dataset= dataset,
                                                                    train_loader = train_loader,
                                                                    )
    best_arch = max(history, key=lambda x: x[0])[1]
    best_score = max(history, key=lambda x: x[0])[0]
    #print(best_arch, " -> score:", best_score)

    index = api.query_index_by_arch(best_arch.tostr()) 

    csv_dict = {'Dataset': dataset, 'Network_index': index, 'Metric': best_score, 'Accuracy': api.get_more_info(int(index), dataset, hp="200", is_random=False)["test-accuracy"] , 'Time': sum(total_times)}

    df_dict = pd.DataFrame([csv_dict])
    result = pd.concat([result, df_dict], ignore_index=True)

    result.to_csv(f'outKERNELwithoutMOD15runs{dataset}.csv', mode='a', index=False, header=False )
    result = pd.DataFrame(columns=['Dataset', 'Network_index', 'Metric', 'Accuracy', 'Time'])

# REA with Synflow without modifications


In [ ]:
def random_topology_func(op_names, max_nodes=4):
  # Return a random architecture
  def random_architecture():
      genotypes = []
      for i in range(1, max_nodes):
        xlist = []
        for j in range(i):
          node_str = "{:}<-{:}".format(i, j)
          op_name = random.choice(op_names)
          xlist.append((op_name, j))
        genotypes.append(tuple(xlist))
      
      return CellStructure(genotypes)
      
  return random_architecture

def mutate_topology_func(op_names):
    """Computes the architecture for a child of the given parent architecture.
    The parent architecture is cloned and mutated to produce the child architecture. 
    The child architecture is mutated by randomly switch one operation to another.
    """
    def mutate_architecture(parent_arch):
        child_arch = deepcopy(parent_arch)
        node_id = random.randint(0, len(child_arch.nodes) - 1)
        node_info = list(child_arch.nodes[node_id])
        snode_id = random.randint(0, len(node_info) - 1)
        xop = random.choice(op_names)
        while xop == node_info[snode_id][0]:
            xop = random.choice(op_names)
        node_info[snode_id] = (xop, node_info[snode_id][1])
        child_arch.nodes[node_id] = tuple(node_info)
        return child_arch

    return mutate_architecture
model = { "arch": None, "synflow": None}
child = { "arch": None, "synflow": None}

def regularized_evolution_synflow(
    cycles,
    population_size,
    sample_size,
    random_arch,
    mutate_arch,
    api,
    dataset,
    train_loader,
    device
):

    population = []
    hist, total_time_cost = (
        [],
        [],
    )  # Not used by the algorithm, only used to report results.
    current_best_index = []
    # Initialize the population with random models.
    #print('inizio:\n')
    while len(population) < population_size:
        model["arch"] = random_arch()
        #print(model)
        model["synflow"], total_cost = synflow( ###### 
            model["arch"], dataset, train_loader, device, #if use_proxy else api.full_train_epochs
        )
        # Append the info
        population.append(copy.deepcopy(model))
        hist.append((model["synflow"], model["arch"]))
        total_time_cost.append(total_cost)
        current_best_index.append(
            api.query_index_by_arch(max(hist, key=lambda x: x[0])[1]) ############
        )
    ## population has the same values repeated in it
    #print(population)
    #print(total_time_cost)
    # Carry out evolution in cycles. Each cycle produces a model and removes another.
    while len(hist) < cycles:
        # Sample randomly chosen models from the current population.
        start_time, sample = time.time(), []
        while len(sample) < sample_size:
            # Inefficient, but written this way for clarity. In the case of neural
            # nets, the efficiency of this line is irrelevant because training neural
            # nets is the rate-determining step.
            candidate = random.choice(list(population))
            #print(candidate)
            sample.append(candidate) #sample is a list of dicts

        # The parent is the best model in the sample.
        #print(sample) ##always the same cause of population
        parent = max(sample, key=lambda i: i["synflow"]) ## adjust the sample to take the maximum
        #print(type(parent))
        # Create the child model and store it.
        child["arch"] = mutate_arch(parent["arch"])
        child["synflow"], total_cost = synflow( ###### 
            child["arch"], dataset, train_loader, device, #if use_proxy else api.full_train_epochs
        )
        # Append the info
        population[:-1] = population[1:] ##popleft
        population[-1] = child
        hist.append((child["synflow"], child["arch"]))
        current_best_index.append(
            api.query_index_by_arch(max(hist, key=lambda x: x[0])[1]) ############
        )
        total_time_cost.append(total_cost)

    return hist, current_best_index, total_time_cost

search_space = get_search_spaces("tss", "nats-bench") 
random_arch = random_topology_func(op_names = search_space) 
mutate_arch = mutate_topology_func(search_space)


## To obtain RESULTS

In [ ]:
Ktemp = []
for dataset in datasets:
  for run in range(15):
    train_loader = get_datasets(dataset)
    ## REA che ritorna  history e current_best_index
    history, current_best_index, total_times = regularized_evolution_synflow(cycles = 200, ## in history = ((acc, arch), (acc2, arch2)....)
                                                                    population_size = 50, 
                                                                    sample_size = 10, 
                                                                    random_arch= random_arch, 
                                                                    mutate_arch=mutate_arch,  
                                                                    api=api, 
                                                                    dataset= dataset,
                                                                    train_loader = train_loader,
                                                                    device=device)
    best_arch = max(history, key=lambda x: x[0])[1]
    best_score = max(history, key=lambda x: x[0])[0]
    #print(best_arch, " -> score:", best_score)

    index = api.query_index_by_arch(best_arch.tostr()) 

    csv_dict = {'Dataset': dataset, 'Network_index': index, 'Metric': best_score, 'Accuracy': api.get_more_info(int(index), dataset, hp="200", is_random=False)["test-accuracy"] , 'Time': sum(total_times)}

    df_dict = pd.DataFrame([csv_dict])
    result = pd.concat([result, df_dict], ignore_index=True)

    result.to_csv(f'outSYNFLOWwithoutMOD15runs{dataset}.csv', mode='a', index=False, header=False )
    result = pd.DataFrame(columns=['Dataset', 'Network_index', 'Metric', 'Accuracy', 'Time'])

# OPERATIONS to do on child architecture during "modified mutation"

In [ ]:
def mutation_without_skip(child_arch):
  names = ['nor_conv_1x1', 'nor_conv_3x3', 'avg_pool_3x3', 'none']
  node_id = random.randint(0, len(child_arch.nodes) - 2) #escludo dalla mutation il nodo 2
  node_info = list(child_arch.nodes[node_id])
  snode_id = random.randint(0, len(node_info) - 1)
  xop = random.choice(names)
  while xop == node_info[snode_id][0]:
      xop = random.choice(names)
  
  node_info[snode_id] = (xop, node_info[snode_id][1])
  child_arch.nodes[node_id] = tuple(node_info)
  return child_arch

def inversion(child_arch, op_names):
  #print(child_arch)
  if(child_arch.nodes[2][1][0].find('skip') == 0):
    node_info = list(child_arch.nodes[2])
    #print(node_info[0])
    temp = node_info[0]
    node_info[0] = (child_arch.nodes[2][1][0], 0)
    node_info[1] = (temp[0], 1)
    child_arch.nodes[2]= tuple(node_info)
    #child_arch.nodes[2][1] = tuple(temp)
    return child_arch
  else:
    node_info = list(child_arch.nodes[2])
    #print(node_info[0])
    temp = node_info[0]
    node_info[0] = (child_arch.nodes[2][2][0], 0)
    node_info[2] = (temp[0], 2)
    child_arch.nodes[2]= tuple(node_info)
    #child_arch.nodes[2][1] = tuple(temp)
    return child_arch

def first_node(child_arch):
  #skip al node_id0 da spostare alla pos 0 del nodo 2 e inversion
    node2_info = list(child_arch.nodes[2])
    node0_info = list(child_arch.nodes[0])
    temp2 = node2_info[0]
    node2_info[0] = (child_arch.nodes[0][0][0], 0)
    node0_info[0] = (temp2[0], 0)
    child_arch.nodes[2]= tuple(node2_info)
    child_arch.nodes[0]= node0_info
    return child_arch
  
def second_node(child_arch):
    #skip al node_id1 nodo da spostare alla pos 0 del nodo 2 (prima invertire con pos 1 del node_id1 ) e crossover
    if(child_arch.nodes[1][1][0].count('skip') == 1):
      info = list(child_arch.nodes[1])
      t0 = info[0]
      #print(info, t0, child_arch.nodes[1][1][0])
      info[0] = (child_arch.nodes[1][1][0], 0)
      info[1] = (t0[0], 1)
      #print(info)
      child_arch.nodes[1]=tuple(info)

    node2_info = list(child_arch.nodes[2])
    node1_info = list(child_arch.nodes[1])
    temp2_0 = node2_info[0]
    temp2_1 = node2_info[1]
    #print(temp2_0, temp2_1)
    node2_info[0] = (child_arch.nodes[1][0][0], 0)
    node2_info[1] = (child_arch.nodes[1][1][0], 1)
    node1_info[0] = temp2_0
    node1_info[1] = temp2_1
    #print(node2_info, node1_info)
    child_arch.nodes[2]= tuple(node2_info)
    child_arch.nodes[1]= tuple(node1_info)
    return child_arch

# REA with NASWOT with condition on SKIP_CONNECTION
Considering a minibatch of data $X = \{  {x_{i} \} }_{i=1}^{N}$ and a network, we can define an indicator variable $c_{i}$ that form a binary code that corresponds to a linear region. We can use the hamming distance between two data points $d_{H}(c_{i},c_{j})$ to measure how dissimilar the two points are. The hamming distance is the minimum number of changes needed to turn one binary code in the other. Or we could also say it is the number of positions where the two codes differ. The correspondance between binary codes can then be calculated by $K_{H}$ with $N_{A}$ as the number of ReLu activations in the network.

$$
K_{H} = \begin{bmatrix} 
    N_{A} - d_{H}(c_{1},c_{1}) & \dots & N_{A} - d_{H}(c_{1},c_{N}) \\
    \vdots & \ddots & \vdots \\
    N_{A} - d_{H}(c_{N},c_{1}) &  \dots      & N_{A} - d_{H}(c_{N},c_{N}) 
    \end{bmatrix}
$$

The scoring metric $s$ is therefore derived from the logarithm of the Kernel norm at initialization. 

$$
s = \log \lVert K_{H} \rVert
$$


In [ ]:
def random_naswotmod_func(op_names, max_nodes=4):
  # Return a random architecture
  def random_m_architecture():
      genotypes = []
      found = False
      while found == False:
        for i in range(1, max_nodes):
            xlist = []
            for j in range(i):
                op_name = random.choice(op_names)
                xlist.append((op_name, j))
            genotypes.append(tuple(xlist))

        if (CellStructure(genotypes).tostr().count('skip')==1): #### controllo skip in 
          found=True
        else:
          genotypes = []
      
      return CellStructure(genotypes)
      
  return random_m_architecture

def mutate_naswotmod_func(op_names):
    """Computes the architecture for a child of the given parent architecture.
    The parent architecture is cloned and mutated to produce the child architecture. 
    The child architecture is mutated by randomly switch one operation to another.
    """
    def mutate_m_architecture(parent_arch):
        child_arch = deepcopy(parent_arch)
        ## three ways:
        if(child_arch.nodes[2][0][0].find('skip') == 0):## if is in node 2 in pos 0: mutation without add other skip
          return mutation_without_skip(child_arch)
        elif (child_arch.nodes[2][1][0].find('skip') ==0 or child_arch.nodes[2][2][0].find('skip')== 0):## if is in node 2 but NOT in pos 0: inversion with pos 1
          return inversion(child_arch, op_names)
        else: ## if skip is NOT in node 2 -> crossover (child_arch is a cellStructure)
          arch_to_str = CellStructure.tostr(child_arch).split('+') #list of elements in each nodes
          if(arch_to_str[0].count('skip') == 1): #skip nel primo nodo
            return first_node(child_arch)
          
        return second_node(child_arch)

    return mutate_m_architecture

model = { "arch": None, "metric": None}
child = { "arch": None, "metric": None}


## P/S of: 100/2, 100/50, 20/20, 100/25, 54/16 ; best is 100/25
def regularized_evolution_kernel_m(
    cycles,
    population_size,
    sample_size,
    random_arch,
    mutate_arch,
    api,
    dataset,
    train_loader
):

    population = []
    history, total_time_cost = (
        [],
        [],
    )  # Not used by the algorithm, only used to report results.
    current_best_index = []
    # Initialize the population with random models.
    while len(population) < population_size:
        model["arch"] = random_arch()
        model["metric"], total_cost = kernel( ###### 
            model["arch"], dataset, batch_size, api, train_loader #if use_proxy else api.full_train_epochs
        )
        # Append the info
        population.append((model["metric"], model["arch"]))
        history.append((model["metric"], model["arch"]))
        #print(history)
        # print(max(history, key=lambda x: x[0]))
        total_time_cost.append(total_cost)
        current_best_index.append(
            api.query_index_by_arch(max(history, key=lambda x: x[0])[1])
        )
    #print(total_time_cost)
    #print("pop: \n", population)
    
    # Carry out evolution in cycles. Each cycle produces a model and removes another.
    while len(history) < cycles:
        # Sample randomly chosen models from the current population.
        start_time, sample = time.time(), []
        while len(sample) < sample_size:
            # Inefficient, but written this way for clarity. In the case of neural
            # nets, the efficiency of this line is irrelevant because training neural
            # nets is the rate-determining step.
            candidate = random.choice(list(population))
            sample.append(candidate)
        # The parent is the best model in the sample.
        #print("sample:\n", sample)
        parent = max(sample, key=lambda i: i[0])
        #print("p:\n", parent[1], parent[0])
        # Create the child model and store it.
        child["arch"] = mutate_arch(parent[1])
        #print("c:\n", child['arch'])
        child["metric"], total_cost = kernel( ###### 
            child["arch"], dataset, batch_size, api, train_loader #if use_proxy else api.full_train_epochs
        )
        #print("c:\n", child['arch'], child['metric'])
        # Append the info
        population[:-1] = population[1:] ##popleft
        population.append((child["metric"], child["arch"]))
        #print('pop after:\n', population)
        history.append((child["metric"], child["arch"]))
        current_best_index.append(
            api.query_index_by_arch(max(history, key=lambda x: x[0])[1])
        )
        total_time_cost.append(total_cost)
        del sample, candidate, parent

    return history, current_best_index, total_time_cost

search_space = get_search_spaces("tss", "nats-bench") 
random_naswotmod = random_naswotmod_func(op_names = search_space) 
mutate_naswotmod = mutate_naswotmod_func(search_space)

## To obtain RESULTS

In [ ]:
Ktemp = []
for dataset in datasets:
  for run in range(15):
    train_loader = get_datasets(dataset)
    ## REA che ritorna  history e current_best_index
    history, current_best_index, total_times = regularized_evolution_kernel_m(cycles = 200, ## in history = ((acc, arch), (acc2, arch2)....)
                                                                    population_size = 50, 
                                                                    sample_size = 10, 
                                                                    random_arch= random_naswotmod, 
                                                                    mutate_arch=mutate_naswotmod,  
                                                                    api=api, 
                                                                    dataset= dataset,
                                                                    train_loader = train_loader,
                                                                    )
    best_arch = max(history, key=lambda x: x[0])[1]
    best_score = max(history, key=lambda x: x[0])[0]
    #print(best_arch, " -> score:", best_score)

    index = api.query_index_by_arch(best_arch.tostr()) 

    csv_dict = {'Dataset': dataset, 'Network_index': index, 'Metric': best_score, 'Accuracy': api.get_more_info(int(index), dataset, hp="200", is_random=False)["test-accuracy"] , 'Time': sum(total_times)}

    df_dict = pd.DataFrame([csv_dict])
    result = pd.concat([result, df_dict], ignore_index=True)

    result.to_csv(f'outKERNELwithoutMOD15runs{dataset}.csv', mode='a', index=False, header=False )
    result = pd.DataFrame(columns=['Dataset', 'Network_index', 'Metric', 'Accuracy', 'Time'])

# REA with Synflow with condition on SKIP_CONNECTION

In [ ]:
def random_synflowmod_func(op_names, max_nodes=4):
  # Return a random architecture
  def random_m_architecture():
      genotypes = []
      found = False
      while found == False:
        for i in range(1, max_nodes):
            xlist = []
            for j in range(i):
                op_name = random.choice(op_names)
                xlist.append((op_name, j))
            genotypes.append(tuple(xlist))

        if (CellStructure(genotypes).tostr().count('skip')==1): #### controllo skip in 
          found=True
        else:
          genotypes = []
      
      return CellStructure(genotypes)
      
  return random_m_architecture

def mutate_synflowmod_func(op_names):
    """Computes the architecture for a child of the given parent architecture.
    The parent architecture is cloned and mutated to produce the child architecture. 
    The child architecture is mutated by randomly switch one operation to another.
    """
    def mutate_m_architecture(parent_arch):
        child_arch = deepcopy(parent_arch)
        ## three ways:
        if(child_arch.nodes[2][0][0].find('skip') == 0):## if is in node 2 in pos 0: mutation without add other skip
          return mutation_without_skip(child_arch)
        elif (child_arch.nodes[2][1][0].find('skip') ==0 or child_arch.nodes[2][2][0].find('skip')== 0):## if is in node 2 but NOT in pos 0: inversion with pos 1
          return inversion(child_arch, op_names)
        else: ## if skip is NOT in node 2 -> crossover (child_arch is a cellStructure)
          arch_to_str = CellStructure.tostr(child_arch).split('+') #list of elements in each nodes
          if(arch_to_str[0].count('skip') == 1): #skip nel primo nodo
            return first_node(child_arch)
          
        return second_node(child_arch)

    return mutate_m_architecture

model = { "arch": None, "synflow": None}
child = { "arch": None, "synflow": None}


## P/S of: 100/2, 100/50, 20/20, 100/25, 54/16 ; best is 100/25
def regularized_evolution_synflow_m(
    cycles,
    population_size,
    sample_size,
    random_arch,
    mutate_arch,
    api,
    dataset,
    train_loader,
    device
):

    population = []
    history, total_time_cost = (
        [],
        [],
    )  # Not used by the algorithm, only used to report results.
    current_best_index = []
    # Initialize the population with random models.
    while len(population) < population_size:
        model["arch"] = random_arch()
        model["synflow"], total_cost = synflow( ###### 
            model["arch"], dataset, train_loader, device, #if use_proxy else api.full_train_epochs
        )
        # Append the info
        population.append(copy.deepcopy(model))
        history.append((model["synflow"], model["arch"]))
        total_time_cost.append(total_cost)
        current_best_index.append(
            api.query_index_by_arch(max(history, key=lambda x: x[0])[1]) ############
        )
    ## population has the same values repeated in it
    #print(population)
    #print(total_time_cost)
    # Carry out evolution in cycles. Each cycle produces a model and removes another.
    while len(history) < cycles:
        # Sample randomly chosen models from the current population.
        start_time, sample = time.time(), []
        while len(sample) < sample_size:
            # Inefficient, but written this way for clarity. In the case of neural
            # nets, the efficiency of this line is irrelevant because training neural
            # nets is the rate-determining step.
            candidate = random.choice(list(population))
            #print(candidate)
            sample.append(candidate) #sample is a list of dicts

        # The parent is the best model in the sample.
        #print(sample) ##always the same cause of population
        parent = max(sample, key=lambda i: i["synflow"]) ## adjust the sample to take the maximum
        #print(type(parent))
        # Create the child model and store it.
        child["arch"] = mutate_arch(parent["arch"])
        child["synflow"], total_cost = synflow( ###### 
            child["arch"], dataset, train_loader, device, #if use_proxy else api.full_train_epochs
        )
        # Append the info
        population[:-1] = population[1:] ##popleft
        population[-1] = child
        history.append((child["synflow"], child["arch"]))
        current_best_index.append(
            api.query_index_by_arch(max(history, key=lambda x: x[0])[1]) ############
        )
        total_time_cost.append(total_cost)
        del sample, candidate, parent

    return history, current_best_index, total_time_cost

search_space = get_search_spaces("tss", "nats-bench") 
random_synflowmod = random_synflowmod_func(op_names = search_space) 
mutate_synflowmod = mutate_synflowmod_func(search_space)

## To obtain RESULTS

In [ ]:
Ktemp = []
for dataset in datasets:
  for run in range(15):
    train_loader = get_datasets(dataset)
    ## REA che ritorna  history e current_best_index
    history, current_best_index, total_times = regularized_evolution_synflow_m(cycles = 200, ## in history = ((acc, arch), (acc2, arch2)....)
                                                                    population_size = 50, 
                                                                    sample_size = 10, 
                                                                    random_arch= random_synflowmod, 
                                                                    mutate_arch=mutate_synflowmod,  
                                                                    api=api, 
                                                                    dataset= dataset,
                                                                    train_loader = train_loader,
                                                                    device=device)
    best_arch = max(history, key=lambda x: x[0])[1]
    best_score = max(history, key=lambda x: x[0])[0]
    #print(best_arch, " -> score:", best_score)

    index = api.query_index_by_arch(best_arch.tostr()) 

    csv_dict = {'Dataset': dataset, 'Network_index': index, 'Metric': best_score, 'Accuracy': api.get_more_info(int(index), dataset, hp="200", is_random=False)["test-accuracy"] , 'Time': sum(total_times)}

    df_dict = pd.DataFrame([csv_dict])
    result = pd.concat([result, df_dict], ignore_index=True)

    result.to_csv(f'outSYNFLOWwithMOD15runs{dataset}.csv', mode='a', index=False, header=False )
    result = pd.DataFrame(columns=['Dataset', 'Network_index', 'Metric', 'Accuracy', 'Time'])